In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
   ManagedOnlineEndpoint,
   ManagedOnlineDeployment,
   Model,
   Environment,
   CodeConfiguration,
)
import os
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from dotenv import load_dotenv

load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [2]:
# General
PROJECT_PATH = os.environ.get("PROJECT_PATH")

# Azure General
AZURE_TENANT_ID = os.environ.get("AZURE_TENANT_ID")
AZURE_SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
AZURE_CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
AZURE_CLIENT_SECRET = os.environ.get("AZURE_CLIENT_SECRET")

# AML
AZURE_ML_RESOURCE_GROUP = os.environ.get("AZURE_ML_RESOURCE_GROUP")
AZURE_ML_WORKSPACE = os.environ.get("AZURE_ML_WORKSPACE")

ENDPOINT_MODEL_NAME = os.environ.get("ENDPOINT_MODEL_NAME")

print(AZURE_TENANT_ID, AZURE_CLIENT_ID, AZURE_ML_RESOURCE_GROUP, AZURE_ML_WORKSPACE)

16b3c013-d300-468d-ac64-7eda0820b6d3 0319a625-aacd-4030-af25-09981016d2f1 rg_aml blx_aml


In [3]:
# credential = DefaultAzureCredential()
# if not credential:
credential = ClientSecretCredential(
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    client_secret=AZURE_CLIENT_SECRET,
)
    
ml_client = MLClient(
   credential, AZURE_SUBSCRIPTION_ID, AZURE_ML_RESOURCE_GROUP, AZURE_ML_WORKSPACE
)

In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%y%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [8]:
# create a blue deployment
model = ml_client.models.get(name="dev.mlflow-sample-model-test_script", version="4")
# model = Model(name="dev.mlflow-sample-model-test_script", version="4")

env = Environment(
    image="crcommon150077523.azurecr.io/aimodelserving/dev.mlflow-sample-model-test_script_v4:2024.7.1.dev20240723T1400",
    inference_config={
        "liveness_route": {"port": 5000, "path": "/version"},
        "readiness_route": {"port": 5000, "path": "/version"},
        "scoring_route": {"port": 5000, "path": "/invocations"},
    },
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    instance_type="Standard_F2s_v2",
    instance_count=1,
)

In [6]:
ml_client.begin_create_or_update(endpoint)

In [9]:
ml_client.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-2407241729987754 exists


ModelException: Error with model: 'NoneType' object has no attribute 'as_posix'